# Machine Learning for Functional Connectivity analysis

This tutorial covers:

* How to download and load COBRE resting state functional datasets.
----------------------------------------------------------------------------

* How to measure connectomes from brain data decomposed to functionally defined regions.
-------------------------------------------------------------------

* SVM based classification of healthy subjects and schizophrenic datasets
-----------------------------------------------------------------------------

All methods are adapted from Nilearn, a Python based toolbox developed for resting state analysis of Brain data. Nilearn is heavily dependent on scikit-learn machine learning library http://scikit-learn.org/stable/, numpy, matplotlib.

In [ ]:
%matplotlib inline

Processing pipeline example for resting state fMRI datasets



Data
----
Fetch COBRE datasets shipped with Nilearn
-----------------------------------------------


In [ ]:
# COBRE datasets release version 0.17 preprocessed using NIAK pipeline. This
# release consists of light weight data with standard preprocessing
# steps used in functional MRI setting.

from nilearn import datasets

# all subjects (146 given in n_subjects argument)
cobre_data = datasets.fetch_cobre(n_subjects=146)
print(cobre_data.keys())


In [ ]:
# Path to functional datasets for each subject
func_imgs = cobre_data.func
print("Paths to data are downloaded and stored in folder "
      "'nilearn_data' in home directory:{0}"
      .format(func_imgs[0]))
print("=============================================")

# Path to confounds: motion, slow drift, compcorr
confounds = cobre_data.confounds
print("Paths to confounds for each subject are also at "
      "stored same location "
      "as functional data:{0}".format(confounds[0]))
print("===============================================")

# Clinical variables are returned as Numpy array but not a path
phenotypes = cobre_data.phenotypic
print(phenotypes[0])

Step 1: Brain Regions of Interest (ROI)
---------------------------------------
We use pre-generated atlas (BASC) to parcellate brain into ROIs. For this, we simply fetch from Nilearn
-----------------------------------


In [ ]:
basc_atlases = datasets.fetch_atlas_basc_multiscale_2015()

# Now, we have atlases of all scales (n=7, 12, 20, 36, 64,
# 122, 197, 325, 444) in the networks.

atlas_img = basc_atlases['scale122']

Visualization: Labelling based brain atlas
------------------------------------------



In [ ]:
from nilearn import plotting

plotting.plot_roi(atlas_img, cmap='Paired', colorbar=True,
                  title='BASC atlas of 122 networks denoted as labels')

Step 2: Timeseries Extraction
-----------------------------
Extract subject specific timeseries signals from brain atlas
--------------------------------------------------------------------


In [ ]:
# For timeseries extraction from functional images, we import
# `NiftiLabelsMasker` from input_data module.

from nilearn.input_data import NiftiLabelsMasker

# We initialize the timeseries extractor by setting standard processing
# parameters required for the extraction.

timeseries_extractor = NiftiLabelsMasker(
    labels_img=atlas_img,  # brain atlas
    smoothing_fwhm=6.,  # Smoothing
    standardize=True, detrend=True, # timeseries signals
    memory='nilearn_cache',  # joblib
    memory_level=2,  # Level of caching
    verbose=2)  # useful to see processing

# We call `fit_transform` on each subject fMRI data.

# `fit` will prepare 2D data matrix from 4D functional images, whereas
# `transform` will filter, extracts and gives out cleaned signals.
# Filtering can be done based on parameters given while initialization
# and from given confounds as given in below argument.

subjects_timeseries = []

for func_img in cobre_data.func:
    signals = timeseries_extractor.fit_transform(func_img)
    subjects_timeseries.append(signals)

Visualization: Extracted timeseries signals (before confounds)
--------------------------------------------------------------



In [ ]:
# We visualize timeseries signals by importing matplotlib

import matplotlib.pyplot as plt

# We show single subject timeseries signals extracted without confounds
plt.figure()
plt.plot(subjects_timeseries[2])
plt.title('Timeseries for single subject extracted from 122 brain regions')
plt.xlabel('Number of regions')
plt.ylabel('Normalized signal')

Timeseries extraction with confounds
------------------------------------



In [ ]:
subjects_timeseries_with_confounds = []

for func_img, confound in zip(func_imgs, confounds):
    signals = timeseries_extractor.fit_transform(func_img,
                                                 confounds=confound)
    subjects_timeseries_with_confounds.append(signals)

Visualization: Extracting timeseries signals (after confounds)
--------------------------------------------------------------



In [ ]:
# We use same matplotlib.pyplot to plot the signals
plt.figure()
plt.plot(subjects_timeseries_with_confounds[2])
plt.title('Timeseries signal (after removing confounds)')
plt.xlabel('Number of regions')
plt.ylabel('Normalized signal')

Step 3: Connectomes estimation
------------------------------



In [ ]:
# Build connectivity matrices using simple 'correlation' on extracted timeseries
# signals

# We import `ConnectivityMeasure` from connectome module and covariance
# estimator LedoitWolf from scikit learn to build connectivity matrices.
from sklearn.covariance import LedoitWolf
from nilearn.connectome import ConnectivityMeasure

# Initialize `ConnectivityMeasure` and measure set in argument 'kind' and
# argument 'cov_estimator'
connectivity = ConnectivityMeasure(kind='correlation',
                                   cov_estimator=LedoitWolf(assume_centered=True))
connectivity_matrices = connectivity.fit_transform(subjects_timeseries_with_confounds)

Visualization: Mean connectome matrix with measure = 'correlation'
------------------------------------------------------------------



In [ ]:
# First, we get the mean of the correlation matrix
mean_correlation_matrix = connectivity_matrices.mean(axis=0)

# Second, visualizing goes here using matplotlib

title = ('Mean over all connectivity matrices with measure "correlation"')
plt.figure()
plt.title(title)
plt.imshow(mean_correlation_matrix, cmap='Paired')
plt.colorbar()
plt.show()

Prediction using Support Vector Classifier
------------------------------------------



In [ ]:
# Till now, we showed you how to measure connectomes using 'correlation'
# measure. Now, we show how to convert connectivity matrices estimated on each
# subject to vector (which contains lower traingular part of those matrices).
# Use these vector coefficients to classify between schizophrenia and
# controls.

# Our classification is based on Support Vector Machine linear classifier
# and using Stratified Shuffle Split cross validation where we train the
# classifier by 75% of the train size and test on the remaining 25%. We repeat
# the same with 100 random splits.

# In this setting, we will be heavily be dependent on scikit-learn library
# which has cool and easy to use machine learning classifiers useful for
# Neuroimaging community.

# For prediction, we convert connectivity matrices to vector, define cross
# validator and use cross_val_score to predict scores for 100 splits.

Symmetric matrices to vector
----------------------------



In [ ]:
from nilearn.connectome import sym_to_vec

connectivity_coefs = sym_to_vec(connectivity_matrices)
# Now, we have connectivity coefficients of all subjects

Cross Validator - for Prediction
---------------------------------



In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

n_iter = 100  # Number of splits
class_type = 'subject_type'  # from phenotypes in cobre data return
classes = np.asarray(phenotypes[class_type])
_, classes_binary = np.unique(classes, return_inverse=True)
cv = StratifiedShuffleSplit(n_splits=n_iter,
                            test_size=0.25, random_state=0)

Now, we predict using SVC
-------------------------



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

svc = LinearSVC(penalty='l2', random_state=0)
iter_for_prediction = cv.split(func_imgs, classes_binary)
scores = []
# Try cv.get_n_splits()
for index, (train_index, test_index) in enumerate(iter_for_prediction):
    prediction_scores = cross_val_score(estimator=svc,  # classifier
                                        X=connectivity_coefs,  # Data to fit
                                        y=classes_binary,  # Target variables
                                        scoring='roc_auc',  # scoring
                                        cv=[(train_index, test_index)],
                                        )
    scores.append(prediction_scores)  # for each split we gather scores

scores = np.asarray(scores)
print(" -- Support Vector Classification -- ")
print("Classification scores '%s': %1.2f +/- %1.2f" % ('correlation',
                                                       scores.mean(),
                                                       scores.std()))

Comparing connectomes using another Tangent Embedding
------------------------------------------------------



In [ ]:
# We try to improve classification accuracy by using different measure
# We repeat the same steps as above from Step 3 but using now kind='tangent' to
# see if we can improve our classification accuracy
connectivity = ConnectivityMeasure(kind='tangent',
                                   cov_estimator=LedoitWolf(assume_centered=True))
# Compute connectome matrices
connectivity_matrices = connectivity.fit_transform(subjects_timeseries_with_confounds)

Visualization: Mean connectome matrix with measure = 'tangent'
---------------------------------------------------------------



In [ ]:
# First, we get the mean of the tangent matrix from attribute 'mean_'
mean_tangent_matrix = connectivity.mean_

# Second, visualizing goes here using matplotlib

title = ('Mean over all connectivity matrices with measure "tangent"')
plt.figure()
plt.title(title)
plt.imshow(mean_tangent_matrix, cmap='Paired')
plt.colorbar()
plt.show()

Prediction using Tangent space
------------------------------



In [ ]:
# Convert connectivity matrices to connectivity vectors
connectivity_coefs2 = sym_to_vec(connectivity_matrices)

# Use them again for classification by using the same cross validator
iter_for_prediction2 = cv.split(func_imgs, classes_binary)
scores2 = []

for index, (train_index, test_index) in enumerate(iter_for_prediction2):
    prediction_scores2 = cross_val_score(estimator=svc,  # classifier
                                         X=connectivity_coefs2,  # Data to fit
                                         y=classes_binary,  # Target variables
                                         scoring='roc_auc',  # scoring
                                         cv=[(train_index, test_index)],
                                         )
    scores2.append(prediction_scores2)  # for each split we gather scores

scores2 = np.asarray(scores2)
print(" -- Support Vector Classification with Tangent Space measure -- ")
print("Classification scores '%s': %1.2f +/- %1.2f" % ('tangent',
                                                       scores2.mean(),
                                                       scores2.std()))